In [1]:
# Useful starting lines
import random

import numpy as np

from test_utils import test

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [2]:
from sklearn import datasets

# Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1))
X[:, :-1] = Xx
print('(N, D) =', X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

In [3]:
def hinge_loss(z):
    return np.clip(1 - z, 0, np.inf)


def calculate_primal_objective(y, X, w, lambda_):
    """Compute the full cost (the primal objective,
    equation (1) in the exercise pdf), that is loss plus regularizer.

    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)

    Returns:
        scalar, non-negative

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_primal_objective(y_test, x_test, w_test, 1)
    49.0
    """
    return np.sum(hinge_loss(y * (X @ w))) + lambda_ * np.linalg.norm(w)**2 / 2

In [4]:
test(calculate_primal_objective)

✅ Your `calculate_primal_objective` passed 4 tests.


In [5]:
def calculate_accuracy(y, X, w):
    """Compute the accuracy on the given dataset (X, y) using the model w.

    Args:
        X: the full dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)

    Returns:
        scalar, between 0 and 1

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_accuracy(y_test, x_test, w_test)
    0.5
    """
    y_pred = (X @ w > 0) * 2 - 1
    return np.mean(y_pred == y)

In [6]:
test(calculate_accuracy)

✅ Your `calculate_accuracy` passed 4 tests.


## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

Subgradient:
$$
N\cdot \nabla \ell\left(y_nx_n^Tw\right) + \lambda w
$$

Where:
$$
\nabla \ell\left(y_nx_n^Tw\right) =
\begin{cases}
-y_nx_n &\quad 1-y_nx_n^Tw > 0\\
0 &\quad \text{otherwise}
\end{cases}
$$

In [7]:
def hinge_loss_subgradient(y, X, w):
    if y * (X @ w) < 1:
        return -y * X.T
    return np.zeros(w.size)


def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """Compute the stochastic gradient of loss plus regularizer.

    Args:
        X: the dataset matrix, shape = (num_examples, num_features)
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        w: shape = (num_features)
        lambda_: positive scalar number
        n: the index of the (one) datapoint we have sampled
        num_examples: N

    Returns:
        numpy array, shape = (num_features)

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1, 2, 3], [4, 5, 6]])
    >>> w_test = np.array([0, 1, 5])
    >>> calculate_stochastic_gradient(y_test, x_test, w_test, 1, 1, 2)
    array([ 8, 11, 17])
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum,
    # not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    return num_examples * hinge_loss_subgradient(y_n, x_n, w) \
        + lambda_ * w

In [8]:
test(calculate_stochastic_gradient)

✅ Your `calculate_stochastic_gradient` passed 4 tests.


Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [9]:
def sgd_for_svm_demo(y, X):
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)
    # big because scales with N due to the formulation of the problem
    # (not an averaged loss)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)

    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0, num_examples - 1)

        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma / (it + 1) * grad

        if it % 20000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print(f'iteration={it}, cost={cost}')

    print('training accuracy =', calculate_accuracy(y, X, w))


sgd_for_svm_demo(y, X)

iteration=0, cost=44625835.47062729
iteration=20000, cost=350.30504875566817
iteration=40000, cost=151.6143938893121
iteration=60000, cost=134.46370257773302
iteration=80000, cost=134.74643398322925
iteration=100000, cost=129.95422695895286
iteration=120000, cost=141.05390287289418
iteration=140000, cost=162.48913686131675
iteration=160000, cost=124.84490658271864
iteration=180000, cost=125.59041521187663
training accuracy = 0.9050966608084359


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [10]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """Compute a coordinate update (closed form) for coordinate n.

    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        lambda_: positive scalar number
        alpha: vector of dual coordinates, shape = (num_examples)
        w: vector of primal parameters, shape = (num_features)
        n: the coordinate to be updated

    Returns:
        w: updated vector of primal parameters, shape = (num_features)
        alpha: updated vector of dual parameters, shape = (num_examples)

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([-0.3, -0.3, -0.3])
    >>> alpha_test = np.array([.1, .1])
    >>> calculate_coordinate_update(y_test, x_test, 1, alpha_test, w_test, 0)
    (array([-0.1,  0.1,  0.3]), array([0.3, 0.1]))
    """        
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])

    g = 1 - y_n * x_n @ w

    if g != 0:
        alpha[n] = min(max(old_alpha_n + lambda_ * g / (x_n.T.dot(x_n)), 0), 1)

        # compute the corresponding update on the primal vector w
        w += 1 / lambda_ * (alpha[n] - old_alpha_n) * y_n * x_n
    return w, alpha

In [11]:
test(calculate_coordinate_update)

✅ Your `calculate_coordinate_update` passed 5 tests.


In [12]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """Calculate the objective for the dual problem.

    Args:
        y: the corresponding +1 or -1 labels, shape = (num_examples)
        X: the dataset matrix, shape = (num_examples, num_features)
        w: vector of primal parameters, shape = (num_features)
        alpha: vector of dual coordinates, shape = (num_examples)
        lambda_: non negative scalar number

    Output:
        scalar

    >>> y_test = np.array([1, -1])
    >>> x_test = np.array([[1., 2., 3.], [4., 5., 6.]])
    >>> w_test = np.array([-0.3, -0.3, -0.3])
    >>> alpha_test = np.array([.1, .1])
    >>> calculate_dual_objective(y_test, x_test, w_test, alpha_test, 1)
    0.065
    """
    return np.sum(alpha) - lambda_ / 2 * np.sum(w**2)

In [13]:
test(calculate_dual_objective)

✅ Your `calculate_dual_objective` passed 5 tests.


In [14]:
# Notice that the gap is going to 0
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2 * int(1e5)
    lambda_ = int(1e4)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)

    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0, num_examples - 1)

        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)

        if it % 20000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f' % (
                    it, primal_value, dual_value, duality_gap))
    print('training accuracy =', calculate_accuracy(y, X, w))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:542.45050, dual:0.00175, gap:542.44875
iteration=20000, primal:211.41002, dual:56.82772, gap:154.58230
iteration=40000, primal:160.44451, dual:88.62388, gap:71.82063
iteration=60000, primal:127.62954, dual:103.75866, gap:23.87087
iteration=80000, primal:124.37902, dual:111.49870, gap:12.88032
iteration=100000, primal:129.15442, dual:115.58985, gap:13.56457
iteration=120000, primal:123.72520, dual:117.91847, gap:5.80673
iteration=140000, primal:123.01412, dual:119.51606, gap:3.49807
iteration=160000, primal:122.59203, dual:120.59775, gap:1.99428
iteration=180000, primal:123.94790, dual:121.06935, gap:2.87854
training accuracy = 0.9244288224956063
